In [13]:
import pandas as pd
import numpy as np

In [2]:
datos = pd.read_parquet('../data/datos_produccion.parquet')

In [18]:
sorted(list(datos.familia.unique()))

['COADYUVANTE',
 'ESPECIALIDADES',
 'FUNGICIDA',
 'HERBICIDA',
 'INSECTICIDA',
 'PRODUCTO TECNICO']

In [23]:
# vamos a sacar el porcentaje de productos termiandos
familia = 'HERBICIDA'
(
    datos
    .query('familia == @familia')
    .assign(terminado=lambda x: (x.fabricado >= x.programado).astype(int))
    .terminado.mean()
)

0.375

In [69]:
# vamos a encontrar el mejor producto
familia = 'HERBICIDA'
filtered_data = datos.query('familia == @familia')
min_fecha = filtered_data.fecha.min()
max_fecha = filtered_data.fecha.max()
aux = filtered_data.query('@min_fecha <= fecha <= @max_fecha')

(
    aux
    .groupby('sku')
    .agg(
        fabricado=pd.NamedAgg('fabricado', 'sum'),
        programado=pd.NamedAgg('programado', 'sum')
    )
    .assign(cumplimiento=lambda x: round(x.fabricado / x.programado*100, 2))
    .sort_values('cumplimiento', ascending=False)
    .dropna()
    .head(1).T.to_dict()
)

{'H209': {'fabricado': 1000.0, 'programado': 1000.0, 'cumplimiento': 100.0}}